# Arbeiten mit Umgebungen


Wenn Sie ein Skript als Azure Machine Learning-Auftrag ausführen, müssen Sie den Ausführungskontext für die Auftragsausführung festlegen. Eine Schlüsselkonfiguration ist das Computeziel, auf dem das Skript ausgeführt wird. Dies kann die lokale Arbeitsstation (in diesem Fall die Compute-Instanz) oder ein Remotecomputeziel wie der verwaltete Azure Machine Learning-Computecluster sein, der bedarfsgesteuert bereitgestellt wird.

In diesem Notebook erstellen Sie einen Computecluster und erkunden Computeziele für Aufträge.

## Vorbereitung

Sie benötigen die neueste Version des Pakets **azureml-ai-ml**, um den Code in diesem Notebook auszuführen. Führen Sie die folgende Zelle aus, um zu überprüfen, ob das Paket installiert ist.

> **Hinweis**:
> Wenn das Paket **azure-ai-ml** nicht installiert ist, führen Sie `pip install azure-ai-ml` aus, um es zu installieren.

In [ ]:
## Herstellen einer Verbindung mit Ihrem Arbeitsbereich

Sie können nun eine Verbindung mit Ihrem Arbeitsbereich herstellen, nachdem Sie die erforderlichen SDK-Pakete installiert haben.

Um eine Verbindung mit einem Arbeitsbereich herzustellen, benötigen Sie Bezeichnerparameter: eine Abonnement-ID, einen Ressourcengruppennamen und einen Arbeitsbereichsnamen. Der Ressourcengruppenname und Arbeitsbereichsname sind bereits für Sie ausgefüllt. Sie müssen lediglich die Abonnement-ID angeben, um den Befehl zu vervollständigen.

Um die erforderlichen Parameter zu finden, klicken Sie rechts oben in Studio auf das Abonnement und den Namen des Arbeitsbereichs. Rechts wird ein Bereich geöffnet.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Kopieren Sie die Abonnement-ID, und ersetzen Sie **YOUR-SUBSCRIPTION-ID** durch den Wert, den Sie kopiert haben. </p>

## Ausführen eines Skripts als Auftrag

Um ein Modell zu trainieren, erstellen Sie zuerst im Ordner **src** das Skript **diabetes_training.py**. Das Skript nutzt die Datei **diabetes.csv** im Ordner, in dem sich die Trainingsdaten befinden.

Beachten Sie, dass Bibliotheken am Anfang des Skripts importiert werden. Funktionen in diesen Bibliotheken dienen zum Verarbeiten der Daten und Trainieren des Modells. In der Compute-Instanz, die Sie zur Ausführung des Skripts nutzen, müssen diese Bibliotheken installiert sein.

In [ ]:
Nachdem Sie das Skript erstellt haben, können Sie es als Auftrag ausführen. Das Skript verwendet allgemeine Bibliotheken. So können Sie eine kuratierte Umgebung verwenden, die u. a. pandas, numpy und scikit-learn enthält.

Der Auftrag verwendet die neueste Version der kuratierten Umgebung: `AzureML-sklearn-0.24-ubuntu18.04-py37-cpu`.

In [ ]:
Während der Ausführung des Auftrags können Sie bereits die nächsten Zellen ausführen.

## Auflisten der Umgebungen

Lassen Sie uns die Umgebungen innerhalb des Arbeitsbereichs erkunden. 

Im vorherigen Auftrag haben Sie eine der kuratierten Umgebungen genutzt. Um alle Umgebungen zu erkunden, die bereits im Arbeitsbereich vorhanden sind, können Sie die Umgebungen auflisten: 

In [ ]:
Hinweis: Die Namen aller kuratierten Umgebungen beginnen stets mit **AzureML-** (dieses Präfix können Sie nicht für Ihre eigenen Umgebungen verwenden).

Um eine bestimmte Umgebung zu überprüfen, können Sie sie anhand ihres Namens und ihrer Version abrufen. Sie können beispielsweise die *Beschreibung* und *Tags* der kuratierten Umgebung abrufen, die Sie für den vorherigen Auftrag verwendet haben:

## Erstellen und Verwenden einer benutzerdefinierten Umgebung

Wenn eine kuratierte Umgebung nicht alle Python-Pakete enthält, die Sie zum Ausführen Ihres Skripts benötigen, können Sie eine eigene benutzerdefinierte Umgebung erstellen. Indem Sie alle erforderlichen Pakete in einer Umgebung auflisten, können Sie Ihre Skripts problemlos erneut ausführen. Alle Abhängigkeiten werden in der Umgebung gespeichert, die Sie dann unabhängig von der verwendeten Compute-Instanz in der Auftragskonfiguration angeben können.

Beispielsweise können Sie eine Umgebung einfach aus einem Docker-Image erstellen. Für bestimmte Frameworks wie PyTorch gibt es ein öffentliches Docker-Image, das bereits alles enthält, was Sie benötigen. 

Lassen Sie uns eine Umgebung aus einem Docker-Image erstellen:

In [ ]:
Die Umgebung ist jetzt in Ihrem Arbeitsbereich registriert, und Sie können darauf verweisen, wenn Sie ein Skript als Auftrag ausführen:

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Der Auftrag schlägt allerdings schnell fehl! Überprüfen Sie die Fehlermeldung. </p>

Sie erfahren, dass es kein Modul namens „pandas“ gibt. Für einen solchen Fehler gibt es zwei mögliche Ursachen:

- Das Skript verwendet pandas, aber die Bibliothek wurde nicht importiert (`import pandas as pd`). 
- Das Skript importiert die Bibliothek am Anfang des Skripts, aber in der Compute-Instanz wurde die Bibliothek nicht installiert (`pip install pandas`).

Nach Überprüfung des Skripts `diabetes-training.py` stellen Sie fest, dass das Skript stimmt, was bedeutet, dass die Bibliothek nicht installiert wurde. Mit anderen Worten, die Umgebung enthält nicht die erforderlichen Pakete.

Erstellen Sie eine neue Umgebung unter Verwendung des Docker-Basisimages aus dem vorherigen Auftrag. Nun fügen Sie eine Conda-Spezifikation hinzu, um sicherzustellen, dass die erforderlichen Pakete installiert werden. Führen Sie zunächst die folgende Zelle aus, um die Conda-Spezifikationsdatei zu erstellen:

Beachten Sie, dass alle erforderlichen Abhängigkeiten in der Conda-Spezifikationsdatei enthalten sind, damit das Skript erfolgreich ausgeführt werden kann.

Erstellen Sie eine neue Umgebung mithilfe des Docker-Basisimages **und** der Conda-Spezifikationsdatei, um die erforderlichen Abhängigkeiten hinzuzufügen. Azure Machine Learning erstellt die Conda-Umgebung aufsetzend auf dem von Ihnen bereitgestellten Docker-Image. 

In [ ]:
Jetzt können Sie einen Auftrag mit der neuen Umgebung übermitteln, um das Skript auszuführen:

Durch Übermittlung des Auftrags mit der neuen benutzerdefinierten Umgebung wird der Build der Umgebung ausgelöst. Wenn Sie zum ersten Mal eine neu erstellte Umgebung verwenden, kann der Build der Umgebung 10–15 Minuten dauern. Dies bedeutet auch, dass ihr Auftrag länger dauert. 

Sie können den Build der Umgebung auch manuell auslösen, ehe Sie einen Auftrag übermitteln. Der Build der Umgebung muss nur dann erfolgen, wenn Sie sie zum ersten Mal verwenden. 

Wenn der Auftrag den Build einer neuen Umgebung auslöst, können Sie die Protokolle der Erstellung auf der Registerkarte **Ausgaben und Protokolle** des Auftrags überprüfen. Öffnen Sie **azureml-logs/20_image_build_log.txt**, um die Protokolle des Builds der Umgebung zu überprüfen. 

![Screenshot der Buildprotokolle](./images/screenshot-logs.png)

In [ ]:
env = ml_client.environments.get("AzureML-sklearn-0.24-ubuntu18.04-py37-cpu", version=44)
print(env. description, env.tags)

## Create and use a custom environment

If a curated environment doesn't include all the Python packages you need to run your script, you can create your own custom environment. By listing all necessary packages in an environment, you can easily re-run your scripts. All the dependencies are stored in the environment which you can then specify in the job configuration, independent of the compute you use.

For example, you can create an environment simply from a Docker image. Certain frameworks like PyTorch will have a public Docker image that already includes everything you need. 

Let's create an environment from a Docker image:

In [ ]:
from azure.ai.ml.entities import Environment

env_docker_image = Environment(
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    name="docker-image-example",
    description="Environment created from a Docker image.",
)
ml_client.environments.create_or_update(env_docker_image)

The environment is now registered in your workspace and you can reference it when you run a script as a job:

In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="docker-image-example:1",
    compute="aml-cluster",
    display_name="diabetes-train-custom-env",
    experiment_name="diabetes-training"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> The job will quickly fail! Review the error message. </p>

The error message will tell you that there is no module named pandas. There are two possible causes for such an error:

- The script uses pandas but didn't import the library (`import pandas as pd`). 
- The script does import the library at the top of the script but the compute didn't have the library installed (`pip install pandas`).

After reviewing the `diabetes-training.py` script you can observe the script is correct, which means the library wasn't installed. In other words, the environment didn't include the necessary packages.

Let's create a new environment, using the base Docker image used in the previous job. Now, you'll add a conda specification to ensure the necessary packages will be installed. First, run the following cell to create the conda specification file:

In [ ]:
%%writefile src/conda-env.yml
name: basic-env-cpu
channels:
  - conda-forge
dependencies:
  - python=3.7
  - scikit-learn
  - pandas
  - numpy
  - matplotlib

Note that all necessary dependencies are included in the conda specification file for the script to run successfully.

Create a new environment using the base Docker image **and** the conda specification file to add the necessary dependencies. Azure Machine Learning will build the conda environment on top of the Docker image you provided. 

In [ ]:
from azure.ai.ml.entities import Environment

env_docker_conda = Environment(
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    conda_file="./src/conda-env.yml",
    name="docker-image-plus-conda-example",
    description="Environment created from a Docker image plus Conda environment.",
)
ml_client.environments.create_or_update(env_docker_conda)

Now, you can submit a job with the new environment to run the script:

In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="docker-image-plus-conda-example:1",
    compute="aml-cluster",
    display_name="diabetes-train-custom-env",
    experiment_name="diabetes-training"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Submitting the job with the new custom environment triggers the build of the environment. The first time you use a newly created environment, it can take 10-15 minutes to build the environment, which also means your job will take longer to complete. 

You can also choose to manually trigger the build of the environment before you submit a job. The environment only needs to be built the first time you use it. 

When the job triggers the build of a new environment, you can review the logs of the build in the **Outputs + logs** tab of the job. Open **azureml-logs/20_image_build_log.txt** to inspect the logs of the environment build. 

![Screenshot build logs](./images/screenshot-logs.png)